In [17]:
import networkx as nx
import plotly.graph_objects as go
import json

# JSON 파일 경로
json_file = 'shape_meas.json'  # JSON 파일의 경로를 넣어주세요.

print("JSON 파일 경로 설정 완료")

# JSON 파일 읽기
with open(json_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

print("JSON 파일 읽기 완료")
print("데이터 내용: ", data)

# 그래프 생성
G = nx.DiGraph()

print("그래프 생성 완료")

# 노드 추가 함수 (재귀적으로 자식 노드들을 추가)
def add_nodes_from_json(data, parent=None):
    for node_id, node_info in data.items():
        if isinstance(node_info, dict):
            node_name = node_info.get("name", "Unnamed Node")
            print(f"노드 추가: {node_id} - {node_name}")
            G.add_node(node_id, label=node_name)  # 노드 추가
            if parent:
                G.add_edge(parent, node_id)  # 부모 노드와 자식 노드 연결
                print(f"엣지 추가: {parent} -> {node_id}")
            
            # 자식 노드들이 있으면 재귀적으로 추가
            children = node_info.get("children", {})
            add_nodes_from_json(children, parent=node_id)
        else:
            # 노드 정보가 문자열인 경우 (leaf 노드)
            print(f"노드 추가: {node_id} - {node_info}")
            G.add_node(node_id, label=node_info)  # 노드 추가
            if parent:
                G.add_edge(parent, node_id)  # 부모 노드와 자식 노드 연결
                print(f"엣지 추가: {parent} -> {node_id}")

# JSON 데이터를 그래프에 추가
add_nodes_from_json(data)

print("노드 및 엣지 추가 완료")

# 노드 레이블 설정
labels = nx.get_node_attributes(G, "label")
print("노드 레이블 설정 완료")

# 노드와 엣지 위치 설정
pos = nx.spring_layout(G, seed=42)

# 엣지 추가
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color='#888'),
    hoverinfo='none',
    mode='lines'
)

# 노드 추가
node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=[labels[node] for node in G.nodes()],
    textposition="bottom center",
    marker=dict(
        size=10,
        color='lightgreen',
        line=dict(width=2)
    )
)

# 레이아웃 설정
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='초등 수학 지식공간',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False),
                    yaxis=dict(showgrid=False, zeroline=False)
                ))

fig.show()
print("그래프 시각화 완료")


JSON 파일 경로 설정 완료
JSON 파일 읽기 완료
데이터 내용:  {'03': {'name': '도형과 측정', 'children': {'03-01': {'name': '입체도형', 'children': {'03-01-02': {'name': '직육면체의 모양', 'children': {'03-01-02-01': '직육면체', '03-01-02-02': '직육면체의 구성요소와 성질', '03-01-02-03': '정육면체', '03-01-02-04': '정육면체의 구성요소와 성질', '03-01-02-05': '직육면체의 겨냥도', '03-01-02-06': '직육면체의 전개도', '03-01-02-07': '정육면체의 겨냥도', '03-01-02-08': '정육면체의 전개도', '03-01-02-09': '직육면체 겉넓이 구하기', '03-01-02-10': '정육면체 겉넓이 구하기', '03-01-02-11': '직육면체의 부피 구하기', '03-01-02-12': '정육면체의 부피 구하기'}}, '03-01-03': {'name': '원기둥의 모양', 'children': {'03-01-03-01': '원기둥', '03-01-03-02': '원뿔', '03-01-03-03': '원기둥의 구성요소와 성질', '03-01-03-04': '원뿔의 구성요소와 성질', '03-01-03-05': '원기둥 전개도'}}, '03-01-04': {'name': '구의 모양', 'children': {'03-01-04-01': '구 이해하기', '03-01-04-02': '구의 구성요소와 성질'}}, '03-01-05': {'name': '여러 모양으로 쌓기', 'children': {'03-01-05-01': '모양에 대해 위치나 방향을 이용하여 말하기', '03-01-05-02': '입체도형의 쌓기나무 개수 구하기', '03-01-05-03': '입체도형의 위. 앞. 옆에서 본 모양 표현하기', '03-01-05-04': '입체도형의 모양 추론하기'}}, '03

그래프 시각화 완료


In [1]:
import networkx as nx
import plotly.graph_objects as go
import json
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# JSON 파일 경로
json_file = 'shape_meas.json'  # JSON 파일의 경로를 넣어주세요.

print("JSON 파일 경로 설정 완료")

# JSON 파일 읽기
with open(json_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

print("JSON 파일 읽기 완료")
print("데이터 내용: ", data)

# 그래프 생성
G = nx.DiGraph()

print("그래프 생성 완료")

# 노드 추가 함수 (재귀적으로 자식 노드들을 추가)
def add_nodes_from_json(data, parent=None):
    for node_id, node_info in data.items():
        if isinstance(node_info, dict):
            node_name = node_info.get("name", "Unnamed Node")
            print(f"노드 추가: {node_id} - {node_name}")
            G.add_node(node_id, label=node_name)  # 노드 추가
            if parent:
                G.add_edge(parent, node_id)  # 부모 노드와 자식 노드 연결
                print(f"엣지 추가: {parent} -> {node_id}")
            
            # 자식 노드들이 있으면 재귀적으로 추가
            children = node_info.get("children", {})
            add_nodes_from_json(children, parent=node_id)
        else:
            # 노드 정보가 문자열인 경우 (leaf 노드)
            print(f"노드 추가: {node_id} - {node_info}")
            G.add_node(node_id, label=node_info)  # 노드 추가
            if parent:
                G.add_edge(parent, node_id)  # 부모 노드와 자식 노드 연결
                print(f"엣지 추가: {parent} -> {node_id}")

# JSON 데이터를 그래프에 추가
add_nodes_from_json(data)

print("노드 및 엣지 추가 완료")

# 노드 레이블 설정
labels = nx.get_node_attributes(G, "label")
print("노드 레이블 설정 완료")

# 형태소 분석을 통한 비슷한 키워드 연결
def connect_similar_keywords(G, labels):
    nodes = list(G.nodes())
    for i, node1 in enumerate(nodes):
        for node2 in nodes[i+1:]:
            label1 = labels[node1]
            label2 = labels[node2]
            
            # 형태소 분석 후 명사 추출
            nouns1 = set(okt.nouns(label1))
            nouns2 = set(okt.nouns(label2))
            
            # 공통 명사가 있는지 확인
            if nouns1 & nouns2:
                G.add_edge(node1, node2)
                print(f"비슷한 키워드로 엣지 추가: {node1} -> {node2}")

# 비슷한 키워드로 노드 연결
connect_similar_keywords(G, labels)

# 노드와 엣지 위치 설정
pos = nx.spring_layout(G, seed=42)

# 엣지 추가
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color='#888'),
    hoverinfo='none',
    mode='lines'
)

# 노드 추가
node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=[labels[node] for node in G.nodes()],
    textposition="bottom center",
    marker=dict(
        size=10,
        color='lightgreen',
        line=dict(width=2)
    )
)

# 레이아웃 설정
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='초등 수학 지식공간',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False),
                    yaxis=dict(showgrid=False, zeroline=False)
                ))

fig.show()
print("그래프 시각화 완료")


JSON 파일 경로 설정 완료
JSON 파일 읽기 완료
데이터 내용:  {'03': {'name': '도형과 측정', 'children': {'03-01': {'name': '입체도형', 'children': {'03-01-02': {'name': '직육면체의 모양', 'children': {'03-01-02-01': '직육면체', '03-01-02-02': '직육면체의 구성요소와 성질', '03-01-02-03': '정육면체', '03-01-02-04': '정육면체의 구성요소와 성질', '03-01-02-05': '직육면체의 겨냥도', '03-01-02-06': '직육면체의 전개도', '03-01-02-07': '정육면체의 겨냥도', '03-01-02-08': '정육면체의 전개도', '03-01-02-09': '직육면체 겉넓이 구하기', '03-01-02-10': '정육면체 겉넓이 구하기', '03-01-02-11': '직육면체의 부피 구하기', '03-01-02-12': '정육면체의 부피 구하기'}}, '03-01-03': {'name': '원기둥의 모양', 'children': {'03-01-03-01': '원기둥', '03-01-03-02': '원뿔', '03-01-03-03': '원기둥의 구성요소와 성질', '03-01-03-04': '원뿔의 구성요소와 성질', '03-01-03-05': '원기둥 전개도'}}, '03-01-04': {'name': '구의 모양', 'children': {'03-01-04-01': '구 이해하기', '03-01-04-02': '구의 구성요소와 성질'}}, '03-01-05': {'name': '여러 모양으로 쌓기', 'children': {'03-01-05-01': '모양에 대해 위치나 방향을 이용하여 말하기', '03-01-05-02': '입체도형의 쌓기나무 개수 구하기', '03-01-05-03': '입체도형의 위. 앞. 옆에서 본 모양 표현하기', '03-01-05-04': '입체도형의 모양 추론하기'}}, '03

그래프 시각화 완료
